## Notebook to visualize and compare the surface forcing to Wolfe & Cessi (2014)

In [ ]:
import xarray as xr
import xgcm
from matplotlib import pyplot as plt
import nc_time_axis
import numpy as np

#plt.rcParams['axes.facecolor']='black'
#plt.style.use('dark_background')

In [ ]:
path   = "/data/dkamm/nemo_output/DINO/"

## Load domain (not lazy)

In [ ]:
from pathlib import Path
import xnemogcm as xn

In [ ]:
datadir  = Path(path + 'Stratification')

In [ ]:
domcfg = xn.open_domain_cfg(datadir=datadir)

In [ ]:
Data = xn.open_nemo(domcfg=domcfg, files=[
        datadir / 'NEVERWORLD_grid_T.nc',
        datadir / 'NEVERWORLD_grid_U.nc',
        datadir / 'NEVERWORLD_grid_V.nc'
    ])

In [ ]:
Data = Data.assign_coords(dict(
    depth_v=(["z_c", "y_f", "x_c"], - Data.isel(t=-1).e3v.cumsum('z_c').values),
    depth_t=(["z_c", "y_c", "x_c"], - Data.isel(t=-1).e3t.cumsum('z_c').values),
    lon_f=("x_f", domcfg.glamf.values[0,:]-60.),
    lon_t=("x_c", domcfg.glamt.values[0,:]-60.),
    lat_f=("y_f", domcfg.gphif.values[:,0]),
    lat_t=("y_c", domcfg.gphit.values[:,0]),
    )
)

## Wind-forcing

In [ ]:
# Wolfe Cessi (2014) analytical forcing

lat = np.linspace(-5000, 5000, 200)

tau = 0.1 * ( - np.cos(
    3 * np.pi * lat / 2 / (5000) )
    + 0.8 * np.exp( - lat**2 / 640**2)
)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(16,6), sharey=True)
#
Data.utau.isel(t=-1, x_f=10).plot(color='black', x='lat_t', ax=axs[0])
axs[0].set_xlabel('latitude [N°]')
axs[0].fill_between(Data.lat_t, -0.12, 0.22, where=domcfg.umask.isel(x_f=0, z_c=0)==1, facecolor='grey', alpha=.2)
axs[0].margins(x=0, y=0)
axs[0].set_title('DINO configuration')
axs[0].grid()
#
axs[1].plot(lat, tau, color='black')
axs[1].fill_between(lat, -0.12, 0.22, where=lat<-3800, facecolor='grey', alpha=.2)
axs[1].margins(x=0, y=0)
axs[1].set_xlabel('latitude [km]')
axs[1].set_title('Wolfe & Cessi (2014)')
axs[1].grid()

# Restoring temperature profile

In [ ]:
# Wolfe Cessi (2014) analytical forcing

lat = np.linspace(-5000, 5000, 200)

T_star_max = 20.0 * np.cos(np.pi * lat / 2 / (5000) )**2 + 5 * np.exp( - (lat - 5000.)**2 / 1800**2)
T_star_min = 20.0 * np.cos(np.pi * lat / 2 / (5000) )**2

In [ ]:
# DINO analytical forcing
#ztstar(ji,jj)   = (zts_eq - (zts_n + zcos_sais2 * zdeltaT) ) * COS( ( rpi * gphit(ji,jj) ) * z1_2L )**2 + (zts_n + zcos_sais2 * zdeltaT)
#
sign_lat = (Data.lat_t>0) * 1 - (Data.lat_t<0) * 1
#
T_star_max_d = ( 25.0 - ( 1.0 + sign_lat * 2.0) ) * np.cos(np.pi * Data.lat_t / 2 / 70. )**2 + (1.0 + sign_lat * 2.0)
T_star_min_d = ( 25.0 - ( 1.0 - sign_lat * 2.0) ) * np.cos(np.pi * Data.lat_t / 2 / 70. )**2 + (1.0 - sign_lat * 2.0)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(16,6), sharey=True)
#
axs[0].plot(Data.lat_t, T_star_max_d, color='midnightblue', label='June')
axs[0].plot(Data.lat_t, T_star_min_d, color='darkred', label='Dezember')
axs[0].fill_between(Data.lat_t, -1.5, 25.5, where=domcfg.umask.isel(x_f=0, z_c=0)==1, facecolor='grey', alpha=.2)
axs[0].fill_between(Data.lat_t, T_star_min_d, T_star_max_d, facecolor='grey', alpha=.2)
axs[0].margins(x=0, y=0)
axs[0].set_xlabel('latitude [km]')
axs[0].set_title('DINO configuration')
axs[0].grid()
axs[0].legend()
#
axs[1].plot(lat, T_star_max, color='black')
axs[1].plot(lat, T_star_min, color='black')
axs[1].fill_between(lat, -1.5, 25.5, where=lat<-3800, facecolor='grey', alpha=.2)
axs[1].fill_between(lat, T_star_min, T_star_max, facecolor='grey', alpha=.2)
axs[1].margins(x=0, y=0)
axs[1].set_xlabel('latitude [km]')
axs[1].set_title('Wolfe & Cessi (2014)')
axs[1].grid()

## Zonally averaged density

In [ ]:
Data.isel(t=-1, y_c=slice(1,-1), x_c=slice(1,-1), z_c=slice(0,-1)).soce.mean('x_c').plot()

In [ ]:
Data.isel(t=-1, y_c=slice(1,-1), x_c=slice(1,-1), z_c=0).soce.plot.contour()

In [ ]:
domcfg.gdept_0.isel(y_c=41, z_c=-1, x_c=slice(4,20)).plot(yincrease=False, x='glamt')

In [ ]:
domcfg.gphit.isel(y_c=42).values

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(16,8))
Data['vertical_level'] = (['z_c', 'x_c', 'y_c'], np.mgrid[1:37, 0:60, 0:199][0])
Data.vertical_level.isel(y_c=100).plot.contourf(y='depth_t', levels=36, ax=axs[0])
Data.vertical_level.isel(y_c=41).plot.contourf(y='depth_t', levels=36, ax=axs[1])